In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
import re

from Utilities.EvaluationMain import *
from Utilities.Utilities import ReadYaml, SerializeObjects, DeserializeObjects
from Models.Caller import *
from BatchMIEvaluation import LoadModelConfigs, LoadParams
from Utilities.Visualization import VisReconGivenZ_FCA, HeatMapFreqZ_FCA, VisReconGivenFC_ZA, VisReconExtractZ_FC

def Normalization(series):
    if np.issubdtype(series.dtype, np.number):  # Check if the series is numeric.
        return (series - series.min()) / (series.max() - series.min())  # Apply min-max normalization.
    return series  # Return the series unchanged if it's not numeric.


In [2]:
TabList_ART_500 = ['SKZ_ART_30_500','SKZ_ART_50_500', 'ConVAE_ART_30', 'ConVAE_ART_50', 'SKZFC_ART_30_500', 'SKZFC_ART_50_500', 'TCMIDKZFC_ART_30_500', 'TCMIDKZFC_ART_50_500', 'FACFC_ART_30_500', 'FACFC_ART_50_500']
TabList_ART_800 = ['SKZ_ART_30_800','SKZ_ART_50_800', 'ConVAE_ART_30', 'ConVAE_ART_50', 'SKZFC_ART_30_800', 'SKZFC_ART_50_800', 'TCMIDKZFC_ART_30_800', 'TCMIDKZFC_ART_50_800', 'FACFC_ART_30_800', 'FACFC_ART_50_800']

TabList_II_500 = ['SKZ_II_30_500','SKZ_II_50_500', 'ConVAE_II_30', 'ConVAE_II_50', 'SKZFC_II_30_500', 'SKZFC_II_50_500', 'TCMIDKZFC_II_30_500', 'TCMIDKZFC_II_50_500', 'FACFC_II_30_500', 'FACFC_II_50_500']
TabList_II_800 = ['SKZ_II_30_800','SKZ_II_50_800', 'ConVAE_II_30', 'ConVAE_II_50', 'SKZFC_II_30_800', 'SKZFC_II_50_800', 'TCMIDKZFC_II_30_800', 'TCMIDKZFC_II_50_800', 'FACFC_II_30_800', 'FACFC_II_50_800']    

In [3]:
TableList = os.listdir('./EvalResults/Tables/')
BMtableList = os.listdir('./Benchmarks/EvalResults/Tables/')

AcctableList = [tab for tab in TableList if 'Acc' in tab and 'Nj1.' in tab]
BMAcctableList = [tab for tab in BMtableList if 'Acc' in tab and 'Nj1.' in tab]

AcctableSet = pd.DataFrame() 
for tab in AcctableList:
    Acctables = pd.read_csv('./EvalResults/Tables/'+tab)
    AcctableSet = pd.concat([AcctableSet, Acctables], axis=0)

BMAcctableSet = pd.DataFrame() 
for tab in BMAcctableList:
    Acctables = pd.read_csv('./Benchmarks/EvalResults/Tables/'+tab)
    BMAcctableSet = pd.concat([BMAcctableSet, Acctables], axis=0)

AcctableSet = pd.concat([AcctableSet, BMAcctableSet])


MItableList = [tab for tab in TableList if 'MI' in tab and 'Nj1.' in tab]
BMMItableList = [tab for tab in BMtableList if 'MI' in tab and 'Nj1.' in tab]

MItableSet = pd.DataFrame() 
for tab in MItableList:
    MItables = pd.read_csv('./EvalResults/Tables/'+tab)
    MItableSet = pd.concat([MItableSet, MItables], axis=0)


BMMItableSet = pd.DataFrame() 
for tab in BMMItableList:
    MItables = pd.read_csv('./Benchmarks/EvalResults/Tables/'+tab)
    BMMItableSet = pd.concat([BMMItableSet, MItables], axis=0)

MItableSet = pd.concat([MItableSet, BMMItableSet])

In [4]:
ARTBenchList = [ 'BaseVAE_ART_30',  'TCVAE_ART_30', 'FACVAE_ART_30', 'BaseVAE_ART_50',  'TCVAE_ART_50', 'FACVAE_ART_50']
IIBenchList = ['BaseVAE_II_30', 'TCVAE_II_30', 'FACVAE_II_30', 'BaseVAE_II_50', 'TCVAE_II_50', 'FACVAE_II_50']

# Combine the ART 800 and ART 500 tab lists into one for analysis
AnalTabList = TabList_ART_800 + TabList_ART_500 + TabList_II_500 +  TabList_II_800 + ARTBenchList + IIBenchList

# Define the list of metrics to be analyzed
AnalMetricList= [   '(1) I(V;Z)', 
                    '(2) $I(V; \\acute{Z} \\mid Z)$',
                    '(3) $I(V;\\acute{Z})$',
                    '(4) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
                    '(5) $I(S;\\acute{Z})$',
                    '(6) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$']

# Filter the MI table set for models in the Analysis Tab List and reset the index for clean slicing
AnalMItableSet =  MItableSet[(MItableSet['Model'].isin(AnalTabList))].reset_index(drop=True)
# Filter the Accuracy table set for models in the Analysis Tab List and reset the index for clean slicing
AnalAcctableSet =  AcctableSet[(AcctableSet['Model'].isin(AnalTabList))].reset_index(drop=True)
# Calculate the Reverse Kullback-Leibler Divergence (RevKLD) by taking the exponential of the negative mean KLD results
#AnalAcctableSet['RevKLD'] = np.exp(-AnalAcctableSet['MeanKldRes'])
AnalAcctableSet['MAPEnorm'] = AnalAcctableSet['MAPEnorm']/100
#AnalAcctableSet['NormRevKLD'] = (-AnalAcctableSet['RevKLD'] + AnalAcctableSet['RevKLD'].min()) / (-AnalAcctableSet['RevKLD'].max() + AnalAcctableSet['RevKLD'].min())



# Keep only the 'Model' and 'RevKLD' columns in the Analysis Accuracy table set
AnalAcctableSet = AnalAcctableSet[['Model', 'MeanKldRes', 'MAPEnorm']].copy()
# Filter MItableSet for models in the AnalTabList and reset the index for clean slicing.
AnalMItableSet = MItableSet[(MItableSet['Model'].isin(AnalTabList))].reset_index(drop=True)
# Group the data by 'Model' and 'Metrics', calculate the mean, and reset the index.
AnalMItableSet = AnalMItableSet.groupby(['Model', 'Metrics']).mean().reset_index().copy()
# Filter the rows where 'Metrics' are in the list of analytical metrics and reset the index.
AnalMItableSet = AnalMItableSet[AnalMItableSet['Metrics'].isin(AnalMetricList)].reset_index(drop=True)
# Pivot the table to have 'Model' as rows, 'Metrics' as columns, and 'Values' as cell values.
AnalMItableSet = pd.pivot(AnalMItableSet, index='Model', columns='Metrics', values='Values').reset_index().copy()

# Apply the normalization function to each column in the table.
NormAnalMItableSet = AnalMItableSet.apply(Normalization)
AnalAccMItable = pd.merge(AnalAcctableSet, AnalMItableSet, on='Model', how='inner').sort_values('Model').reset_index(drop=True)

In [5]:
ModelMeta = {}
ModelMap = {'BaseVAE' : '$\\beta$'+'-VAE',
              'ConVAE' : 'ConVAE', 
              'TCVAE' : 'TCVAE', 
              'FACVAE' : 'FACVAE', 
              'SKZ' : 'SKZ',
              'SKZFC' : 'SKZFC',
              'TCMIDKZFC' : 'TCMIDKZFC', 
              'FACFC' : 'FACFC'}

ModelFeature = {
    'BaseVAE':    {'Model':'BaseVAE',   'SKZ': 'O', 'DKZ': '',  'TC': '',  'MI': '',  'Theta': ''},
    'ConVAE':     {'Model':'ConVAE',    'SKZ': 'O', 'DKZ': '',  'TC': '',  'MI': '',  'Theta': 'Δ'},
    'TCVAE':      {'Model':'TCVAE',     'SKZ': '',  'DKZ': 'O', 'TC': 'O', 'MI': 'O', 'Theta': ''},
    'FACVAE':     {'Model':'FACVAE',    'SKZ': 'O', 'DKZ': '',  'TC': 'O', 'MI': '',  'Theta': 'O'},
    'SKZ':        {'Model':'SKZ',       'SKZ': 'O', 'DKZ': '',  'TC': '',  'MI': '',  'Theta': 'Δ'},
    'SKZFC':      {'Model':'SKZFC',     'SKZ': 'O', 'DKZ': '',  'TC': '',  'MI': '',  'Theta': 'O'},
    'TCMIDKZFC':  {'Model':'TCMIDKZFC', 'SKZ': '',  'DKZ': 'O', 'TC': 'O', 'MI': 'O', 'Theta': 'O'},
    'FACFC':      {'Model':'FACFC',     'SKZ': 'O', 'DKZ': '',  'TC': 'O', 'MI': '',  'Theta': 'O'}
}

In [6]:
AggTab = pd.DataFrame()
AnalAccMItable['ModelTitle'] = None
AnalAccMItable['SigType'] = None


for num, row in AnalAccMItable.iterrows():
    Name, SigType = row['Model'].split('_')[:2]
    AnalAccMItable.loc[num, 'ModelTitle'] = Name
    AnalAccMItable.loc[num, 'SigType'] = SigType
    #rows = pd.DataFrame.from_dict(ModelFeature[Name], orient='index').T
    #AggTab = pd.concat([AggTab, rows], axis=0)

SelTable = AnalAccMItable[['ModelTitle','SigType','MeanKldRes', 'MAPEnorm', 
                '(2) $I(V; \\acute{Z} \\mid Z)$','(4) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$', 
                '(6) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$']]
SelTable = SelTable.rename(columns={'(2) $I(V; \\acute{Z} \\mid Z)$':'$I(V; \\acute{Z} \\mid Z)$' , 
                '(4) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '$I(V;\\acute{\\Theta} \\mid \\acute{Z})$', 
                '(6) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$' : '$I(S;\\acute{\\Theta} \\mid \\acute{Z})$'})

GroupMeanTable = SelTable.groupby(['ModelTitle','SigType']).mean().reset_index()
GroupMeanTable = GroupMeanTable.round(3)

# Create a MultiIndex for the columns
Cols = ['MeanKldRes', 'MAPEnorm', '$I(V; \\acute{Z} \\mid Z)$',      
        '$I(V;\\acute{\\Theta} \\mid \\acute{Z})$', '$I(S;\\acute{\\Theta} \\mid \\acute{Z})$']

Iterables = [GroupMeanTable['SigType'].unique(), Cols]

MultIndex = pd.MultiIndex.from_product(Iterables, names=['SigType', 'Measurement'])

# Create a new DataFrame with MultiIndex columns and ModelTitle as index
NestGroupMeanTable = pd.DataFrame(index=GroupMeanTable['ModelTitle'].unique(), columns=MultIndex)

# Populate the new DataFrame
for _, row in GroupMeanTable.iterrows():
    for measurement in Cols:
        NestGroupMeanTable.loc[row['ModelTitle'], (row['SigType'], measurement)] = row[measurement]
NestGroupMeanTable = NestGroupMeanTable.reset_index().copy()
NestGroupMeanTable = NestGroupMeanTable.rename(columns={'index':'Model'})

In [7]:
AggTab = pd.DataFrame()

for num, row in NestGroupMeanTable.iterrows():
    rows = pd.DataFrame.from_dict(ModelFeature[row['Model'].values[0]], orient='index').T
    AggTab = pd.concat([AggTab, rows], axis=0)

AggTab = AggTab.set_index('Model')
AggTab.columns = pd.MultiIndex.from_product([AggTab.columns, ['']], names=['Measurement', 'SigType'])

# Make sure 'Model' is set as an index in NestGroupMeanTable to align with AggTab for merging
NestGroupMeanTable.set_index('Model', inplace=True)

# Perform the merge
AggTab = pd.merge(AggTab, NestGroupMeanTable, left_index=True, right_index=True)

# Reset the index to make 'Model' a column
AggTab.reset_index(inplace=True)
AggTab.to_excel('./EvalResults/Tables/AggTab.xlsx')
AggTab

Model SKZ DKZ TC MI Theta        ART                                    \
                                 MeanKldRes MAPEnorm $I(V; \acute{Z} \mid Z)$   
0    BaseVAE   O                      0.086    0.055                    2.176   
1     ConVAE   O               Δ      0.076    0.056                    0.031   
2      FACFC   O      O        O      0.074    0.061                    1.515   
3     FACVAE   O      O        O      0.093    0.056                     1.98   
4        SKZ   O               Δ      0.509    0.072                      0.5   
5      SKZFC   O               O      0.068    0.059                    1.333   
6  TCMIDKZFC       O  O  O     O      0.053     0.06                    2.443   
7      TCVAE       O  O  O            0.076    0.081                    3.169   

                                                                             \
  $I(V;\acute{\Theta} \mid \acute{Z})$ $I(S;\acute{\Theta} \mid \acute{Z})$   
0                                  NaN                                  NaN   
1                                1.455                                0.036   
2                                0.412                                1.205   
3                                  NaN                                  NaN   
4                                0.012                                0.055   
5                                 0.41                                1.228   
6                                0.407                                0.917   
7                                  NaN                                  NaN   

          II                                    \
  MeanKldRes MAPEnorm $I(V; \acute{Z} \mid Z)$   
0      0.125    0.061                    1.463   
1      0.116    0.064                    0.145   
2      0.235    0.061                    0.644   
3      0.123    0.059                    1.769   
4      1.148    0.066                    0.429   
5      0.272    0.062                    0.412   
6      0.216    0.062                    1.552   
7      1.508    0.077                    0.773   

                                                                             
  $I(V;\acute{\Theta} \mid \acute{Z})$ $I(S;\acute{\Theta} \mid \acute{Z})$  
0                                  NaN                                  NaN  
1                                1.092                                0.036  
2                                0.211                                0.967  
3                                  NaN                                  NaN  
4                                0.005                                0.064  
5                                0.252                                1.096  
6                                0.206                                0.615  
7                                  NaN                                  NaN